In [1]:
import pandas as pd

In [2]:
#naming data input files
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

In [3]:
#reading input data into pandas dataframes
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

In [4]:
# unwanted prefixes and suffixes - note spaces!
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

In [5]:
# pandas replace, as opposed to basic string replace, defaults to using regular expressions
# this matters for strings like "Dr. " with punctuation, and throws a warning
# the flag regex=False fixes this issue
for prefsuf in prefixes_suffixes:
    student_data_df["student_name"] = \
    student_data_df["student_name"].str.replace(prefsuf, "", regex=False)

In [6]:
# Combine the data into a single dataset.
school_data_complete_df = pd.merge(student_data_df, school_data_df, on=["school_name", "school_name"])
school_data_complete_df.head()
# remember that type, size, and budget are school properties - maybe rename?

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [7]:
# not sure why module wants me to use .count() instead of len()
student_count = len(school_data_complete_df)
school_count = len(school_data_df)
total_budget = school_data_df["budget"].sum()
average_math_score = school_data_complete_df["math_score"].mean()
average_reading_score = school_data_complete_df["reading_score"].mean()
print(f"These {school_count} schools have {student_count:,} students.\n"
      f"Their total budget is ${total_budget:,}.\n"
      f"The average math score is {average_math_score:.2f}, "
      f"and the average reading score is {average_reading_score:.2f}.")

These 15 schools have 39,170 students.
Their total budget is $24,649,428.
The average math score is 78.99, and the average reading score is 81.88.


In [8]:
# we're gonna need boolean Pass/Fail values for each student for many computations,
# so i'm gonna add them to the main dataframe we use
passing_math = school_data_complete_df["math_score"] >= 70
passing_reading = school_data_complete_df["reading_score"] >= 70
school_data_complete_df["pass_math"] = passing_math
school_data_complete_df["pass_reading"] = passing_reading
school_data_complete_df["pass_both"] = passing_math & passing_reading
school_data_complete_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,pass_math,pass_reading,pass_both
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,True,False,False
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,False,True,False
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,False,True,False
3,3,Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,False,False,False
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,True,True,True


In [9]:
# the sum of a series of booleans is the count of True's in it
math_pass_count = school_data_complete_df["pass_math"].sum()
reading_pass_count = school_data_complete_df["pass_reading"].sum()
both_pass_count = school_data_complete_df["pass_both"].sum()

# store percentages in variables
passing_math_percentage = math_pass_count / student_count * 100
passing_reading_percentage = reading_pass_count / student_count * 100
overall_passing_percentage = both_pass_count / student_count * 100

# print results for me to see
print(f"{passing_math_percentage:.2f}% of students passed math.\n"
      f"{passing_reading_percentage:.2f}% of students passed reading.\n"
      f"{overall_passing_percentage:.2f}% of students passed both.")

74.98% of students passed math.
85.81% of students passed reading.
65.17% of students passed both.


In [10]:
# new DataFrame (build from a dictionary) contains requested district summary data
district_summary_df = pd.DataFrame(
          [{"Total Schools": school_count,
          "Total Students": student_count,
          "Total Budget": total_budget,
          "Average Math Score": average_math_score,
          "Average Reading Score": average_reading_score,
          "% Passing Math": passing_math_percentage,
         "% Passing Reading": passing_reading_percentage,
        "% Overall Passing": overall_passing_percentage}])
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [11]:
# district summary column formats dictionary, not intended to ever get used again
disumcolforms = {"Total Students": "{:,}", "Total Budget": "${:,.2f}",
                 "Average Math Score": "{:.1f}", "Average Reading Score": "{:.1f}",
                 "% Passing Math": "{:.0f}", "% Passing Reading": "{:.0f}", 
                 "% Overall Passing": "{:.0f}"}
# i disagree with requested choices: percentages should be formatted to end with "%" symbol

In [12]:
# format values in district-summary dataframe for readability, converting numbers to strings
for col_name in disumcolforms:
    district_summary_df[col_name] = \
    district_summary_df[col_name].map(disumcolforms[col_name].format)
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",79.0,81.9,75,86,65


In [ ]:
# code from 4.7.9 in case we need it later; for now, columns already in correct order

# # Reorder the columns in the order you want them to appear.
# new_column_order = ["Total Schools", "Total Students", "Total Budget","Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]

# # Assign district summary df the new column order.
# district_summary_df = district_summary_df[new_column_order]
# district_summary_df

In [13]:
# here we create a dataframe holding the 5 complicated school stats:
# math/reading avg scores, and math/reading/both passing rates
# because average of booleans gives fraction of True's !!
school_avgs_df = school_data_complete_df.groupby("school_name").mean()
school_avgs_df = \
    school_avgs_df[["reading_score", "math_score", "pass_math", "pass_reading", "pass_both"]]
school_avgs_df

,reading_score,math_score,pass_math,pass_reading,pass_both
school_name,,,,,
Bailey High School,81.033963,77.048432,0.666801,0.819333,0.546423
Cabrera High School,83.975780,83.061895,0.941335,0.970398,0.913348
Figueroa High School,81.158020,76.711767,0.659885,0.807392,0.532045
Ford High School,80.746258,77.102592,0.683096,0.792990,0.542899
Griffin High School,83.816757,83.351499,0.933924,0.971390,0.905995
Hernandez High School,80.934412,77.289752,0.667530,0.808630,0.535275
Holden High School,83.814988,83.803279,0.925059,0.962529,0.892272
Huang High School,81.182722,76.629414,0.656839,0.813164,0.535139
Johnson High School,80.966394,77.072464,0.660576,0.812224,0.535392


In [14]:
school_summary_df = pd.merge(school_data_df, school_avgs_df, on=["school_name", "school_name"])
school_summary_df

,School ID,school_name,type,size,budget,reading_score,math_score,pass_math,pass_reading,pass_both
0,0,Huang High School,District,2917,1910635,81.182722,76.629414,0.656839,0.813164,0.535139
1,1,Figueroa High School,District,2949,1884411,81.158020,76.711767,0.659885,0.807392,0.532045
2,2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,0.938671,0.958546,0.898921
3,3,Hernandez High School,District,4635,3022020,80.934412,77.289752,0.667530,0.808630,0.535275
4,4,Griffin High School,Charter,1468,917500,83.816757,83.351499,0.933924,0.971390,0.905995
5,5,Wilson High School,Charter,2283,1319574,83.989488,83.274201,0.938677,0.965396,0.905826
6,6,Cabrera High School,Charter,1858,1081356,83.975780,83.061895,0.941335,0.970398,0.913348
7,7,Bailey High School,District,4976,3124928,81.033963,77.048432,0.666801,0.819333,0.546423
8,8,Holden High School,Charter,427,248087,83.814988,83.803279,0.925059,0.962529,0.892272
9,9,Pena High School,Charter,962,585858,84.044699,83.839917,0.945946,0.959459,0.905405


In [15]:
# here add a derived column for per-capita spending, reindex by school_name,
# and remove numerical indexes from the original schools df
school_summary_df["per_cap"] = school_summary_df["budget"] / school_summary_df["size"]
school_summary_df = school_summary_df.set_index(["school_name"]) \
    [["type", "size", "budget", "per_cap",
     'math_score', 'reading_score', 'pass_math', 'pass_reading', 'pass_both']]
school_summary_df

,type,size,budget,per_cap,math_score,reading_score,pass_math,pass_reading,pass_both
school_name,,,,,,,,,
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.656839,0.813164,0.535139
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.532045
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,0.938671,0.958546,0.898921
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,0.667530,0.808630,0.535275
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.905995
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,0.938677,0.965396,0.905826
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.913348
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,0.666801,0.819333,0.546423
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,0.925059,0.962529,0.892272


In [16]:
school_summary_df.columns

Index(['type', 'size', 'budget', 'per_cap', 'math_score', 'reading_score',
       'pass_math', 'pass_reading', 'pass_both'],
      dtype='object')

In [19]:
# requested output has no name for index column, so
school_summary_df.index.name = None
# over-complicated column renaming, so i can see what i'm changing
# the easy option is df.columns = new_col_names
old_col_names = ['type', 'size', 'budget', 'per_cap', 'math_score', 'reading_score',
                 'pass_math', 'pass_reading', 'pass_both']
new_col_names = ["School Type", "Total Students", "Total School Budget", "Per Student Budget",
                "Average Math Score", "Average Reading Score", "% Passing Math",
                "% Passing Reading", "% Overall Passing"]
col_rename_dic = {}
for i in range(len(old_col_names)):
    col_rename_dic[old_col_names[i]] = new_col_names[i]
col_rename_dic

{'type': 'School Type',
 'size': 'Total Students',
 'budget': 'Total School Budget',
 'per_cap': 'Per Student Budget',
 'math_score': 'Average Math Score',
 'reading_score': 'Average Reading Score',
 'pass_math': '% Passing Math',
 'pass_reading': '% Passing Reading',
 'pass_both': '% Overall Passing'}

In [20]:
# rename columns as requested
school_summary_df.rename(columns = col_rename_dic, inplace = True)
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.656839,0.813164,0.535139
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.532045
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,0.938671,0.958546,0.898921
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,0.667530,0.808630,0.535275
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.905995
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,0.938677,0.965396,0.905826
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.913348
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,0.666801,0.819333,0.546423
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,0.925059,0.962529,0.892272
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,0.945946,0.959459,0.905405


In [21]:
# todo: format outputs, module only asks for budget and per-cap budget, with same format
for colnam in ("Total School Budget", "Per Student Budget"):
    school_summary_df[colnam] = school_summary_df[colnam].map("${:,.2f}".format)
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,0.656839,0.813164,0.535139
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,0.659885,0.807392,0.532045
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,0.938671,0.958546,0.898921
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,0.667530,0.808630,0.535275
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,0.933924,0.971390,0.905995
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,0.938677,0.965396,0.905826
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,0.941335,0.970398,0.913348
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,0.666801,0.819333,0.546423
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,0.925059,0.962529,0.892272
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,0.945946,0.959459,0.905405
